Data Clipping

In [27]:
import numpy as np 
import pandas as pd 
import os
from matplotlib import pyplot as plt
import torch
import cv2
from pathlib import Path

In [39]:
import os
from shapely.geometry import Polygon, box
import json
import pyproj
from rasterio.mask import mask    
import rasterio 
def crs_transform(x,y, input_crs=4326, target_crs=32630):
        transformer = pyproj.Transformer.from_crs(pyproj.CRS.from_epsg(input_crs), pyproj.CRS.from_epsg(target_crs), always_xy=True)
        reprojected_x , reprojected_y = transformer.transform(x, y)
        return reprojected_x, reprojected_y
 
 
def convert_all_points(ls_ls, input_crs=4326, target_crs=32630):
    """ convert points to crs of geotiff image"""
    t_t = []
    for i in ls_ls:
        l = list(crs_transform(i[0], i[1], input_crs, target_crs))
        #l.append(i[2])
        t_t.append(l)
    return t_t
 
 
def get_intersection_percentage(reprojected_polygon, geobox):
    return reprojected_polygon.intersection(geobox).area / reprojected_polygon.area
 

In [85]:
def get_image_with_highest_intersection(geojson_file_path, root_dir):
    with open(geojson_file_path, "r") as f:
        geojson_file = json.load(f)

    #print(geojson_file['features'])
   
    geojson_points_unprojected = geojson_file['features'][0]['geometry']['coordinates'][0]
    #print(geojson_points_unprojected)
 
    #print(geojson_points_unprojected)
    best_match = None
    best_match_percentage = 0
    tif_files = [file for file in Path(root_dir).iterdir() if file.suffix in ['.tif', '.tiff']]
    #print(tif_files)
    # for each image for the geojson file
    for image_path in tif_files:
        #image_path = os.path.join(dir_path, tif_file)
        #print(image_path)
        with rasterio.open(image_path) as src:
            image_geometry = box(*src.bounds)
            crs = f"{src.crs}"
            crs = int(crs.split(":")[-1])
            reprojected_points = convert_all_points(geojson_points_unprojected,target_crs=crs)
           
            reprojected_polygon = Polygon(reprojected_points)
            #print(reprojected_points)
        intersection_percentage = get_intersection_percentage(reprojected_polygon, image_geometry)
        if (intersection_percentage > best_match_percentage) and (intersection_percentage < 0.8):
            best_match_percentage = intersection_percentage
            best_match = image_path

    return geojson_file_path, best_match.as_posix(), best_match_percentage
 


In [86]:
geo_json_path = "/home/idisc02/Saarland_Forest_monitoring_research/Reforestation_Monitoring/Plant_For_Planet_above200_dataset/polygon_7.geojson"
polygon_name = "polygon_7"
# Directory path
root_dir = f"/home/idisc02/Saarland_Forest_monitoring_research/Reforestation_Monitoring/Plant_For_Planet_above200_dataset/{polygon_name}/1b146699-0721-4fc8-a15d-e7ce221cc7fd/PSScene"

geojson_file, match_image_path, _ = get_image_with_highest_intersection(geo_json_path, root_dir)
print(_)


0.787955096883202


In [87]:
def get_masked_region(geojson_file, image_path):
    with open(geojson_file, "r") as f:
        geojson_file = json.load(f)
   
    geojson_points_unprojected = geojson_file['features'][0]['geometry']['coordinates'][0]
 
    with rasterio.open(image_path) as src:
        #image_geometry = box(*src.bounds)
        crs = f"{src.crs}"
        crs = int(crs.split(":")[-1])
       
        # Mask the image
        reprojected_points = convert_all_points(geojson_points_unprojected,target_crs=crs)
           
        reprojected_polygon = Polygon(reprojected_points)
        out_image, out_transform = rasterio.mask.mask(src, [reprojected_polygon], crop=True)
 
        out_meta = src.meta.copy()
 
        out_meta.update({
            "driver": "GTiff",
            "height": out_image.shape[1],
            "width": out_image.shape[2],
            "transform": out_transform
        })
 
        save_path = "/home/idisc02/Saarland_Forest_monitoring_research/Reforestation_Monitoring/Plant_For_Planet_above200_dataset/polygon_7/1b146699-0721-4fc8-a15d-e7ce221cc7fd"
        # Write the masked image to a new file
        with rasterio.open(f'{save_path}/masked_image.tif', 'w', **out_meta) as dest:
            dest.write(out_image)

In [88]:
get_masked_region(geojson_file, match_image_path)